In [1]:
!python --version

Python 3.12.12


In [2]:
!git clone https://github.com/BigMak1/rag_fact_checking.git

Cloning into 'rag_fact_checking'...


remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 40 (delta 6), reused 40 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (40/40), 586.23 KiB | 5.75 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [3]:
!ls -al

total 16
drwxr-xr-x 4 root root 4096 Feb 26 08:40 .
drwxr-xr-x 5 root root 4096 Feb 26 08:38 ..
drwxr-xr-x 5 root root 4096 Feb 26 08:40 rag_fact_checking
drwxr-xr-x 2 root root 4096 Feb 26 08:39 .virtual_documents


In [4]:
!pwd

/kaggle/working


In [5]:
!pip install -q -r rag_fact_checking/DRAGON/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.9/87.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.9/458.9 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.3/508.3 MB 3.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.6/192.6 kB 1.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 102.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 9.6 MB/s eta 0:00

In [6]:
import json
import random
import os

os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"
os.environ["VLLM_LOGGING_LEVEL"] = "ERROR"

from datasets import load_dataset
from langchain_community.llms import VLLM
from langchain_huggingface import HuggingFaceEmbeddings
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer

# from rag_bench import baseline, data, evaluator, results
from rag_fact_checking.DRAGON.rag_bench import baseline, data, evaluator, results

In [7]:
torch.cuda.is_available()

True

In [8]:
HIST_PRIVATE_QA_REPO_ID: str = "ai-forever/hist-rag-bench-private-qa"
HIST_PRIVATE_TEXTS_REPO_ID: str = "ai-forever/hist-rag-bench-private-texts"
RANDOM_SEED: int = 42
EMBEDDER_NAME: str = "ai-forever/FRIDA"
LLM_NAME: str = "bond005/meno-tiny-0.1"

In [9]:
LLM_PROMPT: str = """Проанализируйте заданный контекст и ответьте на вопрос пользователя на основе сведений, предоставленных в этом контексте.
Не давайте никаких объяснений и пояснений к своему ответу. Не пишите ничего лишнего. Не извиняйтесь, не стройте диалог. Выдавайте только ответ и ничего больше.
Отвечайте на русском языке.
Если в заданном контексте нет информации для ответа на вопрос пользователя, то ничего не придумывайте и просто откажитесь отвечать.
"""
print(LLM_PROMPT)

Проанализируйте заданный контекст и ответьте на вопрос пользователя на основе сведений, предоставленных в этом контексте.
Не давайте никаких объяснений и пояснений к своему ответу. Не пишите ничего лишнего. Не извиняйтесь, не стройте диалог. Выдавайте только ответ и ничего больше.
Отвечайте на русском языке.
Если в заданном контексте нет информации для ответа на вопрос пользователя, то ничего не придумывайте и просто откажитесь отвечать.



In [10]:
def get_private_qa_dataset(version):
    return load_dataset(HIST_PRIVATE_QA_REPO_ID, revision=version)


def get_private_texts_dataset(version):
    return load_dataset(HIST_PRIVATE_TEXTS_REPO_ID, revision=version)


def get_public_to_private_texts_mapping(version):
    private_texts_ds = get_private_texts_dataset(version)
    mapping = {}
    for item in private_texts_ds["train"]:
        mapping[item["public_id"]] = item["id"]
    return mapping

In [11]:
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.random.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)

In [12]:
#get public datasets (history ones)
texts_ds, questions_ds, version = data.get_datasets(is_hist=True)
print(f"version = {version}")

Latest texts version: 1.15.0
Latest questions version: 1.15.0


README.md:   0%|          | 0.00/374 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/793k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/542 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/377 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/40.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/600 [00:00<?, ? examples/s]

Loaded texts dataset with 542 texts
Loaded questions dataset with 600 questions
version = 1.15.0


In [13]:
#get private datasets (history ones)
qa_dataset = get_private_qa_dataset(version)
mapping = get_public_to_private_texts_mapping(version)

README.md:   0%|          | 0.00/591 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/646k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/600 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/423 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/797k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/542 [00:00<?, ? examples/s]

In [14]:
# Kaggle FIX
import os                                                                                                                                              
# Добавляем путь с libcuda.so в LIBRARY_PATH (используется линкером при JIT-компиляции)                                                                
os.environ["LIBRARY_PATH"] = "/usr/local/nvidia/lib64:" + os.environ.get("LIBRARY_PATH", "")                                                           
print("LIBRARY_PATH =", os.environ["LIBRARY_PATH"])

LIBRARY_PATH = /usr/local/nvidia/lib64:/usr/local/cuda/lib64/stubs


In [15]:
llm = VLLM(
    model=LLM_NAME,
    tensor_parallel_size=2,  # используем оба GPU
    max_new_tokens=256,
    top_p=0.95,
    temperature=0.3,
    vllm_kwargs={
        "gpu_memory_utilization": 0.45,
        "max_num_batched_tokens": 8192,
        "max_model_len": 4096,
        "disable_log_stats": True,
        "seed": RANDOM_SEED
    },
    disable_log_stats=True,
)
tok = AutoTokenizer.from_pretrained(LLM_NAME)

2026-02-26 08:45:42.290229: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1772095542.509697     101 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1772095542.567307     101 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1772095543.042833     101 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1772095543.042869     101 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1772095543.042872     101 computation_placer.cc:177] computation placer alr

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/499 [00:00<?, ?B/s]

2026-02-26 08:46:26.676153: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1772095586.703205     276 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1772095586.713475     276 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1772095586.743834     276 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1772095586.743878     276 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1772095586.743884     276 computation_placer.cc:177] computation placer alr

(Worker_TP1 pid=302) ERROR 02-26 08:46:55 [fa_utils.py:104] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
(Worker_TP0 pid=301) ERROR 02-26 08:46:55 [fa_utils.py:104] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8


(Worker_TP0 pid=301) <frozen importlib._bootstrap_external>:1301: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
(Worker_TP0 pid=301) <frozen importlib._bootstrap_external>:1301: FutureWarning: The cuda.nvrtc module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.nvrtc module instead.
(Worker_TP1 pid=302) <frozen importlib._bootstrap_external>:1301: FutureWarning: The cuda.cudart module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.runtime module instead.
(Worker_TP1 pid=302) <frozen importlib._bootstrap_external>:1301: FutureWarning: The cuda.nvrtc module is deprecated and will be removed in a future release, please switch to use the cuda.bindings.nvrtc module instead.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 10

In [16]:
llm

VLLM(model='bond005/meno-tiny-0.1', tensor_parallel_size=2, temperature=0.3, top_p=0.95, max_new_tokens=256, vllm_kwargs={'gpu_memory_utilization': 0.45, 'max_num_batched_tokens': 8192, 'max_model_len': 4096, 'disable_log_stats': True, 'seed': 42}, client=Qwen2ForCausalLM()(
  (model): Qwen2Model()(
    (embed_tokens): VocabParallelEmbedding(num_embeddings=75968, embedding_dim=1536, org_vocab_size=151936, num_embeddings_padded=151936, tp_size=2)
    (layers): ModuleList()(
      (0-27): 28 x Qwen2DecoderLayer()(
        (self_attn): Qwen2Attention()(
          (qkv_proj): QKVParallelLinear(in_features=1536, output_features=1024, bias=True, tp_size=2, gather_output=False)
          (o_proj): RowParallelLinear(in_features=768, output_features=1536, bias=False, tp_size=2, reduce_results=True)
          (rotary_emb): RotaryEmbedding(head_size=128, rotary_dim=128, max_position_embeddings=32768, base=1000000.0, is_neox_style=True)(
            (apply_rotary_emb): ApplyRotaryEmb(is_neox_style

In [17]:
embedding_model = HuggingFaceEmbeddings(
    model_name="ai-forever/FRIDA",
    model_kwargs={"trust_remote_code": True},
    encode_kwargs={"batch_size": 16, "prompt": "search_document: "},
    query_encode_kwargs={"prompt": "search_query: "}
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/509 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/823 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [18]:
# HYBRID_RERANK = False
# RERANK_CANDIDATE_K = 20

retrieval = baseline.init_retriever(
    texts_ds,
    embedding_model,
    top_k=5,
    chunk_size=500,
    chunk_overlap=100,
    # hybrid_rerank=HYBRID_RERANK,
    # rerank_candidate_k=RERANK_CANDIDATE_K,
)
generation = baseline.init_generation(retrieval, llm, tok, system_prompt=LLM_PROMPT)


Initializing retriever
> Creating vector store
> Vector store created for 542 documents
> Done for 125.52 seconds
Initializing generation chain
> Done


In [19]:
%%time
%%capture

res = baseline.get_results(
    generation, questions_ds, write_logs=False
)

CPU times: user 56.9 s, sys: 3.04 s, total: 59.9 s
Wall time: 2min 53s


In [20]:
#saving the baseline results

res_path = "./test.json"
results.save(res, res_path)

In [21]:
!ls -al

total 152
drwxr-xr-x 4 root root   4096 Feb 26 08:57 .
drwxr-xr-x 5 root root   4096 Feb 26 08:38 ..
-rw-r--r-- 1 root root    579 Feb 26 08:57 log.txt
drwxr-xr-x 5 root root   4096 Feb 26 08:40 rag_fact_checking
-rw-r--r-- 1 root root 131818 Feb 26 08:57 test.json
drwxr-xr-x 2 root root   4096 Feb 26 08:39 .virtual_documents


In [22]:
demo_public_id = 0
print(questions_ds["train"].filter(lambda it: it["id"] == demo_public_id)[0])
print('\n' + json.dumps(res[demo_public_id], ensure_ascii=False, indent=4))

Filter:   0%|          | 0/600 [00:00<?, ? examples/s]

{'id': 0, 'question': 'Какой регион был определён в качестве приоритетного для Дональда Трампа?'}

{
    "found_ids": [
        197,
        134,
        399,
        115,
        458
    ],
    "model_answer": "Индхо-Тихоокеанский регион"
}


In [23]:
#calculate metrics

evaluation_results = evaluator.evaluate_rag_results(res, qa_dataset, mapping)

In [24]:
_ = evaluation_results.to_table(overall_only=True)

Retrieval Metrics:
+----------+---------+
| Metric   |   Value |
+==========+=========+
| Hit Rate |  0.7997 |
+----------+---------+
| MRR      |  0.7893 |
+----------+---------+

Generation Metrics:
+-----------------+---------+
| Metric          |   Value |
+=================+=========+
| ROUGE-1         |  0.5034 |
+-----------------+---------+
| ROUGE-2         |  0.3143 |
+-----------------+---------+
| ROUGE-L         |  0.4809 |
+-----------------+---------+
| Exact Match     |  0.1250 |
+-----------------+---------+
| Substring Match |  0.1483 |
+-----------------+---------+


In [25]:
_ = evaluation_results.to_table(overall_only=False)

Retrieval Metrics:
+----------+-----------+--------+--------+--------+----------+
| Metric   |   Overall |   cond |     mh |    set |   simple |
+==========+===========+========+========+========+==========+
| Hit Rate |    0.7997 | 0.7733 | 0.7756 | 0.8033 |   0.8467 |
+----------+-----------+--------+--------+--------+----------+
| MRR      |    0.7893 | 0.7999 | 0.7730 | 0.7841 |   0.8000 |
+----------+-----------+--------+--------+--------+----------+

Generation Metrics:
+-----------------+-----------+--------+--------+--------+----------+
| Metric          |   Overall |   cond |     mh |    set |   simple |
+=================+===========+========+========+========+==========+
| ROUGE-1         |    0.5034 | 0.6111 | 0.4798 | 0.4259 |   0.4968 |
+-----------------+-----------+--------+--------+--------+----------+
| ROUGE-2         |    0.3143 | 0.3670 | 0.2860 | 0.2265 |   0.3778 |
+-----------------+-----------+--------+--------+--------+----------+
| ROUGE-L         |    0.4809

## Top-10 Documents By Fact-Check Score
Вычисляет score класса `true` (DeReC verifier) для каждого ответа и агрегирует его по найденным документам (`found_ids`).
Выводит top-10 документов по среднему score и количеству появлений.


In [ ]:
from collections import defaultdict

TOP_N_DOCS = 10

if not (DEREC_VERIFIER_REPO and DEREC_VERIFIER_REPO.strip()):
    print("Set DEREC_VERIFIER_REPO first (cell 'DeReC Verifier (optional)').")
else:
    verifier_tok = AutoTokenizer.from_pretrained(DEREC_VERIFIER_REPO, token=(HF_TOKEN.strip() or None))
    verifier = AutoModelForSequenceClassification.from_pretrained(DEREC_VERIFIER_REPO, token=(HF_TOKEN.strip() or None))
    verifier.eval()
    device = "cuda" if torch.cuda.is_available() else "cpu"
    verifier.to(device)

    q_idx = _build_question_index(questions_ds)
    t_idx = _build_text_index(texts_ds)

    doc_score_sum = defaultdict(float)
    doc_score_cnt = defaultdict(int)
    doc_score_max = defaultdict(float)

    with torch.no_grad():
        for qid, pred in res.items():
            qid_str = str(qid)
            question = q_idx.get(qid_str, {}).get("question", "")
            answer = pred.get("model_answer", "")
            found_ids = pred.get("found_ids", [])
            evidence = "\n".join([t_idx.get(doc_id, "") for doc_id in found_ids])[:4000]

            text = f"claim: question: {question}\nanswer: {answer} [SEP] evidence: {evidence}"
            inp = verifier_tok(text, max_length=512, padding="max_length", truncation=True, return_tensors="pt")
            inp = {k: v.to(device) for k, v in inp.items()}
            probs = torch.softmax(verifier(**inp).logits, dim=-1)[0].cpu().numpy()
            true_score = float(probs[2] if probs.shape[0] >= 3 else probs.max())

            for doc_id in found_ids:
                doc_score_sum[doc_id] += true_score
                doc_score_cnt[doc_id] += 1
                doc_score_max[doc_id] = max(doc_score_max[doc_id], true_score)

    ranked = []
    for doc_id, total in doc_score_sum.items():
        cnt = doc_score_cnt[doc_id]
        avg = total / cnt
        ranked.append((doc_id, avg, cnt, doc_score_max[doc_id]))

    ranked.sort(key=lambda x: (-x[1], -x[2], -x[3]))
    top_docs = ranked[:TOP_N_DOCS]

    print(f"Top-{TOP_N_DOCS} documents by average fact-check true_score")
    for i, (doc_id, avg_score, cnt, max_score) in enumerate(top_docs, 1):
        print(f"{i:02d}. doc_id={doc_id} | avg_score={avg_score:.4f} | occurrences={cnt} | max_score={max_score:.4f}")
